# Score sentiments of each topic on news and tweets side
**Objective**: produce sentiment scores for each topic on news side and tweets side

# Roadmap
1. Build topic_news and topic_tweets docs.

# Steps

In [1]:
"""
Initialization
"""

'''
Standard modules
'''
import os
import pickle
import sqlite3
import time
import codecs
import csv
from pprint import pprint

'''
Analysis modules
'''
import pandas as pd


'''
Custom modules
'''
import config
import utilities

'''
Misc
'''
nb_name = '20171012-daheng-score_topic_docs_sentiment'

## Build topic_news and topic_tweets docs

In [2]:
%%time
'''
Load in topics information pkl
'''
if 1 == 1:
    with open(config.TOPICS_LST_PKL, 'rb') as f:
        topics_lst = pickle.load(f)

CPU times: user 2.89 s, sys: 1.04 s, total: 3.92 s
Wall time: 3.92 s


In [4]:
%%time
"""
For each topic, query db
 - write news_doc into topic_news doc
 - write tweet_text into topic_tweets doc
 
All docs are placed inside config.TOPICS_DOCS_DIR
 - topic_news doc follows name convention: [topic_ind]-[topic_name].news.csv
 - topic_tweets doc follows name convention: [topic_ind]-[topic_name].tweets.csv
"""

if 1 == 1:
    '''
    Define topic_news and topic_tweets doc format
    '''
    csv.register_dialect('topics_docs_line', delimiter='\t', doublequote=True, quoting=csv.QUOTE_ALL)
    
    for topic_ind, topic in enumerate(topics_lst[:1]):
        topic_name = topic['name']
        news_native_ids_lst = topic['news_native_ids_lst']
        tweets_native_ids_lst = topic['tweets_native_ids_lst']
        
        print('{} Topic_name: {}; news_num: {}; tweets_num: {}'.format(topic_ind, topic_name, len(news_native_ids_lst), len(tweets_native_ids_lst)))
        
        with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            '''
            Write topic_news doc
            '''
            query_news = '''
            select news_title, news_collected_time, news_native_id, news_doc from news
            where news_native_id = :news_native_id;'''
            
            output_file = os.path.join(config.TOPICS_DOCS_DIR, '{}-{}.news.csv'.format(topic_ind, topic_name))
            with open(output_file, 'w') as f:
                print('\tWriting topic_news doc ...')
                fieldnames = ('news_native_id', 'news_collected_time', 'news_title', 'news_doc')
                writer = csv.DictWriter(f, fieldnames=fieldnames, dialect='topics_docs_line')
                writer.writeheader()

                for news_native_id in topic['news_native_ids_lst']:
                    cursor.execute(query_news, {'news_native_id': news_native_id})
                    for row in cursor.fetchall():
                        writer.writerow({'news_native_id': row['news_native_id'],
                                         'news_collected_time': row['news_collected_time'],
                                         'news_title': row['news_title'],
                                         'news_doc': row['news_doc']})
                    
            
            '''
            Write topic_tweets doc
            '''
            

0 Topic_name: Hillary_Clinton_email_controversy; news_num: 228; tweets_num: 860564
	Writing topic_news doc ...
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 18.7 ms


In [23]:
query_news = '''
select news_title, news_collected_time, news_native_id, news_doc from news
where news_native_id = :news_native_id;'''

news_native_id = '30275'

with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute(query_news, {'news_native_id': news_native_id})
    for row in cursor.fetchall():
        print(row['news_doc'])

Emails sent by Clinton from her personal account weren't archived in official government records. By Josh Gerstein. 3/2/15 11:04 PM EST. Thousands of emails Hillary Clinton generated as secretary of state were not archived as official government records ...
